In [1]:
import random
import tkinter
import copy
from heapq import *
import time

A* without CSP


In [2]:
'''Definirea clasei Node cu parametrii:

  parent - nodul parinte care contine configuratia precedenta de la care s-a ajuns la configuratia actuala
  path - succesiunea de noduri de la nodul intial la nodul curent
  heuristic - euristica folosita pentru calculul scorului nodului
  x - linia pe care se afla celula apasata pentru a ajunge la configuratia actuala
  y - coloana ...

'''

class Node:
    def __init__(self, parent, path, heuristic, game, moves, x, y):
        self.parent = parent
        self.path = path
        if parent is not None:
            self.path.append(parent)
        self.heuristic = heuristic
        self.game = game
        self.moves = moves
        self.x = x
        self.y = y

    def is_final_good(self):      #verifica daca nodul curent este un nod scop
        return self.game.discovered_squares == self.game.number_of_squares - self.game.number_of_bombs

    def is_final_bad(self):     # verifica daca nodul curent contine o configuratie in care a fost apasata o celula cu mina
        return self.game.bombed is True

    def is_final(self):
        return self.is_final_good() or self.is_final_bad()

    def cost(self):
        if self.heuristic == 1:  #miscarea optima descopera cat mai multe casute
           return self.game.number_of_squares - self.game.discovered_squares
        if self.heuristic == -1:
            return  self.game.discovered_squares

In [3]:
'''
  Obiectele din clasa Minesweeper reprezinta configuratii ale tablei de joc. Parametrii sunt:

  n - numarul de linii
  m - numarul de coloane
  number_of_bombs - numarul de mine de pe tabla
  discovered_squares - numarul de celule deja descoperite
  bombed - statutul tablei in functie daca a fost apasata o mina sau nu
  table - matricea de valori a tablei unde -1 reprezinta o mina iar restul valorilor reprezinta numarul de mine vecine
  cover - matrice care retine ce celule au fost descoperite deja, 0 pentru acoperite si 1 pentru descoperite
  chance -
  remaining_bombs - numarul de mine care nu au fost inca localizate

'''
class Minesweeper:

    def __init__(self, n, m, number_of_bombs, discovered_squares, bombed, table, cover):
        self.n = n
        self.m = m
        self.number_of_bombs = number_of_bombs
        self.number_of_squares = n * m
        self.discovered_squares = discovered_squares
        self.bombed = bombed
        self.table = table
        self.cover = cover

    def print_cover(self):
        for row in self.cover:
            print(row)

    def print_table(self):
        for row in self.table:
            print(row)

    def opened_cell(self, x, y): #metoda prin care o celula este descoperita
        opened_cells = [(x,y)]
        self.cover[x][y] = 1 #celula descoperita
        self.discovered_squares += 1
        if self.table[x][y] == -1:
            self.bombed = True
            return []

        l1 = max(0, x - 1)
        l2 = min(self.n - 1, x + 1)
        c1 = max(0, y - 1)
        c2 = min(self.m - 1, y + 1)


        if self.table[x][y] == 0: #celule fara mine in vecinatate deschid celule din jurul lor
            for i in range(l1, l2 + 1):
                for j in range(c1, c2 + 1):
                    if self.cover[i][j] == 0:
                        opened_cells = opened_cells + self.opened_cell(i, j)
        return opened_cells

    def closed_cell(self, x, y): # (pentru bidirectional search) acopera celule
       closed_cells = [(x,y)]
       self.cover[x][y] = 0
       self.discovered_squares -= 1
       if self.table[x][y] == -1:
          self.bombed = True
          return []
       l1 = max(0, x - 1)
       l2 = min(self.n - 1, x + 1)
       c1 = max(0, y - 1)
       c2 = min(self.m - 1, y + 1)


       if self.table[x][y] == 0: #celule fara bombe inchid celule din jurul lor
            for i in range(l1, l2 + 1):
                for j in range(c1, c2 + 1):
                    if self.cover[i][j] == 1:
                        closed_cells = closed_cells + self.closed_cell(i, j)
       return closed_cells



In [7]:
def generate_game(n, m, number_of_bombs):
    table = [[0 for j in range(m)] for i in range(n)]
    cover_start = [[0 for j in range(m)] for i in range(n)]
    cover_stop = [[1 for j in range(m)] for i in range(n)]
    for bomb in range(number_of_bombs):
        x = random.randint(0, n - 1)
        y = random.randint(0, m - 1)
        while table[x][y] == -1:
            x = random.randint(0, n - 1)
            y = random.randint(0, m - 1)
        cover_stop[x][y] = 0
        table[x][y] = -1
    for x in range(n):
        for y in range(m):
            if table[x][y] == -1:
                l1 = max([0, x - 1])
                l2 = min([n - 1, x + 1])
                c1 = max([0, y - 1])
                c2 = min([m - 1, y + 1])
                for a in range(l1, l2 + 1):
                   for b in range(c1, c2 + 1):
                     if table[a][b] != -1:
                        table[a][b] += 1
    start_game = Minesweeper(n, m, number_of_bombs, 0, False, table, cover_start)
    stop_game = Minesweeper(n, m, number_of_bombs, n * m - number_of_bombs, False, table, cover_stop)
    return start_game, stop_game

In [4]:

def get_key(dict, keys, value):
    for key in keys:
        if dict[key] == value:
            return key
    return None

In [9]:
def a_star(start_node):
    win_paths = []
    open_set = []
    heappush(open_set, (start_node.cost(), start_node.game.cover)) #este utilizata o structura heap pentru a retine nodurile din setul open
    visited_config = {start_node : start_node.game.cover}
    closed_set = []
    start = time.time()
    #start_node.game.print_cover()
    while len(open_set) > 0:
        keys = list(visited_config.keys())
        currentNode = get_key(visited_config, keys, heappop(open_set)[1]) #procesarea nodului cu cel mai bun scor din setul open
        closed_set.append(currentNode.game.cover)
        n = currentNode.game.n
        m = currentNode.game.m
        x = 0
        already_opened = []
        open_set = []
        for i in range(n):
            for j in range(m):

                if currentNode.game.cover[i][j] == 0 and (i,j) not in already_opened: #sunt adaugate in heap nodurile ce pot fi create prin miscari pe tabla actuala
                    newNode = copy.deepcopy(currentNode)
                    newNode.parent = currentNode
                    newNode.path.append(currentNode)
                    newNode.moves = currentNode.moves + 1
                    newNode.x = i
                    newNode.y = j
                    opened_cells = newNode.game.opened_cell(i, j)
                    already_opened = already_opened + opened_cells
                    if newNode.is_final_good():
                        return newNode

                    if newNode.is_final_bad():
                        continue


                    keys = list(visited_config.keys())
                    values = list(visited_config.values())
                    key = get_key(visited_config, keys, newNode.game.cover)
                    if newNode.game.cover in values and newNode.cost() >= key.cost():  #se verifica daca configuratia la care s-a ajuns a mai fost vizitata inainte si a obtinut un scor mai bun
                        continue
                    else:
                        if key is not None:
                            key.parent = newNode.parent
                            key.moves = newNode.moves
                    if newNode.is_final_bad() is False and newNode.game.cover not in closed_set:
                        heappush(open_set, (newNode.cost(), newNode.game.cover))
                        x += 1
                        visited_config.update({newNode: newNode.game.cover})

                        if currentNode.parent != None and currentNode.parent.cost() == currentNode.cost() + 1:
                          newNode.path += [currentNode for x in range(currentNode.cost())]
                          return newNode




        #currentNode.game.print_cover()
        print(currentNode.cost())
        if time.time() - start > 10:
          return currentNode
    return None


In [14]:
game1 = generate_game(9, 9, 10)[0]
#game2 = generate_game(16, 16, 40)
#game3 = generate_game(30, 16, 99)
node = Node(None, [], 1, game1, 0, None, None)
node.game.print_table()
start = time.time()
win = a_star(node)
end = time.time()

print("The time of execution  is :",
      (end-start) * 10**3, "ms")


[-1, 2, 1, -1, 1, 0, 0, 1, -1]
[-1, 2, 1, 1, 1, 0, 1, 2, 2]
[1, 1, 0, 0, 0, 0, 1, -1, 1]
[1, 1, 1, 0, 0, 0, 1, 1, 1]
[1, -1, 1, 0, 0, 0, 0, 0, 0]
[2, 2, 1, 0, 1, 1, 1, 0, 0]
[-1, 2, 0, 0, 1, -1, 1, 0, 0]
[-1, 2, 1, 1, 2, 1, 1, 0, 0]
[1, 1, 1, -1, 1, 0, 0, 0, 0]
81
21
The time of execution  is : 13.61989974975586 ms


In [15]:
i = 1
for node in win.path:
    print("Nodul ", i, node)
    i = i + 1
    node.game.print_cover()
    print('\n')
print("Nodul ", i)
win.game.print_cover()


Nodul  1 <__main__.Node object at 0x7c33ff22d420>
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


Nodul  2 <__main__.Node object at 0x7c33ff9ea8c0>
[0, 0, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 1, 1, 1, 1, 1]


Nodul  3 <__main__.Node object at 0x7c33ff9e8760>
[0, 0, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 0]
[0, 1, 1, 1, 1, 1, 1, 0, 0]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 0, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 0, 1, 1, 1, 1, 1]


Nodul  4 <__main__.Node object at 0x7c33ff9e8760>
[0, 0, 0, 0, 1, 1, 1, 1, 0]
[0, 1, 1, 

Bidirectional A* without CSP

In [16]:
def reachable(cover1, cover2, n, m):
    r = True
    for i in range(n):
      for j in range(m):
        r = r and (not(cover1[i][j] == 1 and cover2[i][j] == 0))
    return r


def same(cover1, cover2, n, m):
    same = True
    for i in range(n):
      for j in range(m):
        same = same and (cover1[i][j] == cover2[i][j])
    return same

In [31]:
def bidir(start_node, stop_node):
    win_paths = []
    open_set = []
    heappush(open_set, (start_node.cost(), start_node.game.cover))
    visited_open_config = {start_node : start_node.game.cover}
    closed_set = []

    open_stop_set = []
    heappush(open_stop_set, (stop_node.cost(), stop_node.game.cover))
    visited_closed_config = {stop_node : stop_node.game.cover}
    closed_stop_set = []
    #start_node.game.print_cover()
    #print(open_set)
    #print(open_stop_set)
    run = 0
    start = time.time()
    while len(open_set) > 0 and len(open_stop_set) > 0:
        #print ("Run ", run)
        run += 1
        #print("Before:")
        keys = list(visited_open_config.keys())
        currentNode = get_key(visited_open_config, keys, heappop(open_set)[1])
        closed_set.append(currentNode.game.cover)
        n = currentNode.game.n
        m = currentNode.game.m
        x = 0
        already_opened = []
        keys = list(visited_closed_config.keys())
        values = list(visited_closed_config.values())
        currentClosedNode = get_key(visited_closed_config, keys, heappop(open_stop_set)[1])
        sm = same(currentNode.game.cover, currentClosedNode.game.cover, n, m)
        if sm:
          #print(currentNode, currentClosedNode)
          return currentNode, currentClosedNode
        #print("Current Open Node:")
        #print(currentNode.game.print_cover())
        #print("Current Closed Node:")
        #print(currentClosedNode.game.print_cover())
        for i in range(n):
            for j in range(m):

                if currentNode.game.cover[i][j] == 0 and currentClosedNode.game.cover [i][j] == 1 and (i,j) not in already_opened:
                    #newGame = Minesweeper(currentNode.game.n, currentNode.game.m, currentNode.game.number_of_bombs,
                               #currentNode.game.discovered_squares, currentNode.game.bombed, currentNode.game.table,
                               #currentNode.game.cover, currentNode.game.chance)
                    #print(newGame)
                    #newNode = Node(currentNode, currentNode.path, currentNode.heuristic, newGame)
                    newNode = copy.deepcopy(currentNode)
                    newNode.parent = currentNode
                    newNode.path.append(currentNode)
                    newNode.moves = currentNode.moves + 1
                    newNode.x = i
                    newNode.y = j
                    #agent.open_cell(newNode, i, j)
                    opened_cells = newNode.game.opened_cell(i, j)
                    already_opened = already_opened + opened_cells

                    r = reachable(newNode.game.cover, currentClosedNode.game.cover, n, m)
                    #print(r)
                    if r is False:
                        continue

                    if newNode.heuristic == 3:
                        newNode.game.recalculate_chance(opened_cells)


                   # print("Got in open")
                    heappush(open_set, (newNode.cost(), newNode.game.cover))
                    x += 1
                    visited_open_config.update({newNode: newNode.game.cover})

        if time.time() - start > 5:
          return  currentNode, currentClosedNode
        currentCover = nsmallest(1, open_set)[0][1]
        keys = list(visited_open_config.keys())
        key = get_key(visited_open_config, keys, currentCover)
        sm = same(currentCover, currentClosedNode.game.cover, n, m)
        if sm:
          #print(key, currentClosedNode)
          return key, currentClosedNode
        closed_stop_set.append(currentClosedNode.game.cover)
        already_closed = []

        #print("After:")
        #print("Current Open Node:")
        #for row in range (n):
          #  print (currentCover[row])
        #print("Current Closed Node:")
        #print(currentClosedNode.game.print_cover())

        for i in range(n):
            for j in range(m):

                if currentClosedNode.game.cover[i][j] == 1 and currentCover[i][j] == 0 and (i,j) not in already_closed:
                    #newGame = Minesweeper(currentNode.game.n, currentNode.game.m, currentNode.game.number_of_bombs,
                               #currentNode.game.discovered_squares, currentNode.game.bombed, currentNode.game.table,
                               #currentNode.game.cover, currentNode.game.chance)
                    #print(newGame)
                    #newNode = Node(currentNode, currentNode.path, currentNode.heuristic, newGame)
                    newNode = copy.deepcopy(currentClosedNode)
                    newNode.parent = currentClosedNode
                    newNode.path.append(currentClosedNode)
                    newNode.moves = currentClosedNode.moves + 1
                    newNode.x = i
                    newNode.y = j
                    #agent.open_cell(newNode, i, j)
                    closed_cells = newNode.game.closed_cell(i, j)
                    already_closed = already_closed + closed_cells
                    r = reachable(currentCover, newNode.game.cover, n, m)
                    #print(r)
                    if newNode.is_final_bad() or r is False:
                        continue



                    if newNode.heuristic == 3:
                        newNode.game.recalculate_chance(closed_cells)

                    #print ("Got in close")
                    heappush(open_stop_set, (newNode.cost(), newNode.game.cover))
                    x += 1
                    visited_closed_config.update({newNode: newNode.game.cover})
        if time.time() - start > 5:
          return  currentNode, currentClosedNode



In [21]:
game1 = generate_game(9, 9, 10)
#game2 = generate_game(16, 16, 40)
#game3 = generate_game(30, 16, 99)
start_node = Node(None, [], 1, game1[0], 0, None, None)
start_node.game.print_table()
stop_node = Node(None, [], -1, game1[1], 0, None, None)

start = time.time()
win_start, win_stop = bidir(start_node, stop_node)
end = time.time()

win_start.path[0].game.print_cover()
win_stop.path[0].game.print_cover()

i = 1
for node in win_start.path:
    print("Nodul ", i, node)
    i = i + 1
    node.game.print_cover()
    print('\n')
print("Nodul ", i)
win_start.game.print_cover()

print(win_stop.path)
win_stop.path.reverse()

i += 1
for node in win_stop.path:
    print("Nodul ", i, node)
    i = i + 1
    node.game.print_cover()
    print('\n')

print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")


[1, 1, 0, 0, 0, 0, 0, 0, 0]
[-1, 1, 0, 0, 0, 0, 0, 1, 1]
[1, 1, 0, 0, 0, 1, 1, 2, -1]
[1, 1, 0, 1, 1, 3, -1, 3, 1]
[-1, 1, 0, 1, -1, 4, -1, 3, 0]
[1, 1, 0, 1, 1, 3, -1, 2, 0]
[1, 1, 2, 1, 1, 1, 1, 1, 0]
[1, -1, 3, -1, 2, 0, 0, 0, 0]
[1, 1, 3, -1, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 0, 1, 1]
[0, 1, 1, 1, 0, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 0, 1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1]
[1, 0, 1, 0, 1, 1, 1, 1, 1]
[1, 1, 1, 0, 1, 1, 1, 1, 1]
Nodul  1 <__main__.Node object at 0x7c33fc75e6e0>
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 

In [45]:
avg_moves = 0
for i in range(10000):
  moves = 0
  game = generate_game(16, 30, 99)[0]
  for x in range(16):
    for y in range(30):
      if game.table[x][y] == 0 and game.cover[x][y] == 0:
        moves += 1
        game.opened_cell(x,y)
  for x in range(16):
    for y in range():
      if game.table[x][y] != -1 and game.cover[x][y] == 0:
        moves += 1
  avg_moves += moves
print(avg_moves/10000)

173.71


New A*

In [4]:
class Node:
    def __init__(self, parent, path, heuristic, game, moves, x, y, mistakes = None):
        self.parent = parent
        self.path = path
        if parent is not None:
            self.path.append(parent)
        self.heuristic = heuristic
        self.game = game
        self.moves = moves
        self.x = x
        self.y = y
        self.mistakes = mistakes

    def is_final_good(self):      #verifica daca nodul curent este un nod scop
        return self.game.discovered_squares == self.game.number_of_squares - self.game.number_of_bombs

    def is_final_bad(self):     # verifica daca nodul curent contine o configuratie in care a fost apasata o celula cu mina
        return self.game.bombed is True

    def is_final(self):
        return self.is_final_good() or self.is_final_bad()

    def cost(self):
        if self.heuristic == 1:  #miscarea optima descopera cat mai multe casute
          return self.game.number_of_squares - self.game.discovered_squares
        if self.heuristic == 2 or self.heuristic == 4 :
          return self.game.number_of_squares - len(self.game.flagged) - self.game.discovered_squares
        if self.heuristic == 3:
          return self.game.remaining_bombs


In [5]:
class Minesweeper:

    def __init__(self, n, m, number_of_bombs, discovered_squares, bombed, table, cover, remaining_bombs, flagged, unint):
        self.n = n
        self.m = m
        self.number_of_bombs = number_of_bombs
        self.number_of_squares = n * m
        self.discovered_squares = discovered_squares
        self.bombed = bombed
        self.table = table
        self.cover = cover
        self.remaining_bombs = remaining_bombs
        self.flagged = flagged
        self.unint = unint


    def print_cover(self):
        for row in self.cover:
            print(row)

    def print_table(self):
        for row in self.table:
            print(row)

    def opened_cell(self, x, y): #metoda prin care o celula este descoperita
        opened_cells = [(x,y)]
        self.cover[x][y] = 1 #celula descoperita
        if self.table[x][y] == -1:
            self.bombed = True
            return []

        self.discovered_squares += 1


        l1 = max(0, x - 1)
        l2 = min(self.n - 1, x + 1)
        c1 = max(0, y - 1)
        c2 = min(self.m - 1, y + 1)


        if self.table[x][y] == 0: #celule fara mine in vecinatate deschid celule din jurul lor
            for i in range(l1, l2 + 1):
                for j in range(c1, c2 + 1):
                    if self.cover[i][j] == 0:
                        opened_cells = opened_cells + self.opened_cell(i, j)
        return opened_cells

    def undiscovered_neighbours(self, x , y):
        l1 = max(0, x - 1)
        l2 = min(self.n - 1, x + 1)
        c1 = max(0, y - 1)
        c2 = min(self.m - 1, y + 1)
        unk = []
        n_mines = []
        for i in range(l1, l2 + 1):
          for j in range(c1, c2 + 1):
            if i == x and j == y:
              continue
            if self.cover[i][j] == 1:
              continue
            if (i,j) not in self.flagged:
               unk.append((i,j))
            else:
               n_mines.append((i,j))
        return unk, n_mines

    def solve(self):
      global the_path
      to_check = []
      prev_cover = [[0 if self.cover[a][b] == 0 else 1 for b in range(self.m)] for a in range(self.n)]
      for i in range(self.n):
        for j in range(self.m):
          if self.cover[i][j] == 0 or (i,j) in self.unint:
            continue
          unk, n_mines = self.undiscovered_neighbours(i,j)
          if len(unk) == 0:
            self.unint.append((i,j))
          if len(unk) == self.table[i][j] - len(n_mines):
             self.flagged += unk
             unk = []
             self.unint.append((i,j))
          else:
             if self.table[i][j] == len(n_mines):
                 for safe in unk:
                    if self.cover[safe[0]][safe[1]] == 0:
                      self.opened_cell(safe[0], safe[1])
                      cov = [[0 if self.cover[a][b] == 0 else 1 for b in range(self.m)] for a in range(self.n) ]
                      add_flags = [flag for flag in self.flagged]
                      the_path.append((cov, [], add_flags))
             else:
                    to_check += unk
      if prev_cover != [[0 if self.cover[a][b] == 0 else 1 for b in range(self.m)] for a in range(self.n)]:
          to_check = self.solve()
      to_check = [i for i in to_check if i not in self.flagged and self.cover[i[0]][i[1]] == 0]
      return to_check



    def sim_solve(self, a, b):
      self.cover[a][b] = 1
      prev_cover = [[0 if self.cover[a][b] == 0 else 1 for b in range(self.m)] for a in range(self.n)]
      new_flagged = []
      new_saves = []
      for i in range(self.n):
        for j in range(self.m):
          if self.cover[i][j] == 0 or (i,j) in self.unint or (i == a and j == b):
            continue
          unk, n_mines = self.undiscovered_neighbours(i,j)
          if len(unk) == 0:
            continue
          if len(unk) == self.table[i][j] - len(n_mines):
            new_flagged += [z for z in unk if z not in new_flagged]
          else:
             if self.table[i][j] == len(n_mines):
                 for safe in unk:
                    if safe not in new_saves:
                      new_saves.append(safe)
      self.cover[a][b] = 0
      for i in range(self.n):
        for j in range(self.m):
          if self.cover[i][j] == 0 or (i,j) in self.unint:
            continue
          l1 = max(0, i - 1)
          l2 = min(self.n - 1, i + 1)
          c1 = max(0, j - 1)
          c2 = min(self.m - 1, j + 1)
          nb_mines = 0
          for l in range(l1, l2 + 1):
            for c in range (c1, c2 + 1):
                if (l,c) in new_flagged or (l,c) in self.flagged:
                  nb_mines += 1
          if nb_mines > self.table[i][j]:
            return [-1], [-1]
      return new_flagged, new_saves



    def calc_weight(self, x, y):
        l1 = max(0, x - 1)
        l2 = min(self.n - 1, y + 1)
        c1 = max(0, x - 1)
        c2 = min(self.m - 1, y + 1)
        w = 0
        for l in range(l1, l2 + 1):
            for c in range(c1, c2 + 1):
                if self.cover[l][c] == 0:
                    continue
                unk, n_mines = self.undiscovered_neighbours(l, c)
                if len(unk) == 0:
                    continue
                chance = (len(unk) - (self.table[l][c] - len(n_mines))) / len(unk)
                if w < chance:
                    w = chance
        return w




In [6]:
def new_a_star(start_node):
  open_set = []
  global the_path
  mistakes = 0
  heappush(open_set, (start_node.cost(), start_node.game.cover))
  visited_config = {start_node : start_node.game.cover}
  closed_set = []
  while len(open_set) > 0:
        keys = list(visited_config.keys())
        print(keys)
        currentNode = get_key(visited_config, keys, heappop(open_set)[1])

        the_path.append(currentNode.game)
        print(currentNode)
        if (currentNode.x, currentNode.y) in currentNode.game.flagged:
             continue
        if currentNode.is_final_bad():
          the_path.append(currentNode.parent.game)
          currentNode.parent.game.flagged.append((currentNode.x, currentNode.y))
          print("IISISS BAD")
          mistakes += 1
          continue
        closed_set.append(currentNode.game.cover)
        n = currentNode.game.n
        m = currentNode.game.m
        print("Table")
        currentNode.game.print_table()
        print("Cover before solving: ")
        currentNode.game.print_cover()
        print("No of discoverd squares before")
        print(currentNode.game.discovered_squares)
        to_check = currentNode.game.solve()
        print ("Cover after solving: ")
        currentNode.game.print_cover()
        print("Cost:")
        print(currentNode.cost())
        print("No of discovered square:")
        print(currentNode.game.discovered_squares)
        if currentNode.game.flagged == currentNode.game.number_of_bombs:
          for i in range(n):
            for j in range(m):
              if currentNode.game.cover[i][j] == 0 and currentNode.game.table[i][j] != -1:
                currentNode.game.opened_cell(i, j)
                the_path.append(currentNode.game)
        if currentNode.is_final_good():
          return currentNode, mistakes

        the_path.append((currentNode.cover, to_check))

        #print("A AJUNS AICI")
        print("Open set", open_set)
        if len(to_check) == 0:
          for x in range(n):
            for y in range(m):
              if (x,y) in currentNode.game.flagged or currentNode.game.cover[x][y] == 1:
                continue
            newNode = copy.deepcopy(currentNode)
            newNode.parent = currentNode
            newNode.path.append(currentNode)
            newNode.moves = currentNode.moves + 1
            newNode.x = x
            newNode.y = y
            opened_cells = newNode.game.opened_cell(x, y)
            while newNode.is_final_bad:
              the_path.append(newNode.game)
              the_path.append(newNode.parent.game)
              newNode = copy.deepcopy(currentNode)
              newNode.parent = currentNode
              newNode.path.append(currentNode)
              newNode.moves = currentNode.moves + 1
              newNode.x = x
              newNode.y = y
              mistakes += 1

            opened_cells = newNode.game.opened_cell(x, y)
            heappush(open_set, (currentNode.cost() - len(flags) - len(saves) - 1, newNode.game.cover))
            visited_config.update({newNode: newNode.game.cover})


        for cell in to_check:
           print("A AJUNS AICI")
           x = cell[0]
           y = cell[1]
           flags, saves = currentNode.game.sim_solve(x, y)
           if flags == [-1]: #plasare imposibila a minelor
            continue
           print("New flags:")
           print(x,y)
           print(flags)
           newNode = copy.deepcopy(currentNode)
           newNode.parent = currentNode
           newNode.path.append(currentNode)
           newNode.moves = currentNode.moves + 1
           newNode.x = x
           newNode.y = y
           opened_cells = newNode.game.opened_cell(x, y)
           newNode.game.print_cover()
           #keys = list(visited_config.keys())
           #values = list(visited_config.values())
           #key = get_key(visited_config, keys, newNode.game.cover)
           print(currentNode.cost())
           heappush(open_set, (currentNode.cost() - len(flags) - len(saves) - 1, newNode.game.cover))
           visited_config.update({newNode: newNode.game.cover})




In [7]:
def new_a_star_v2(start_node):
  open_set = []
  global the_path
  mistakes = 0
  heappush(open_set, (start_node.cost(), start_node.game.cover))
  visited_config = {start_node : start_node.game.cover}
  closed_set = []
  while len(open_set) > 0:
        keys = list(visited_config.keys())
        print(keys)
        currentNode = get_key(visited_config, keys, heappop(open_set)[1])
        cov = [[0 if currentNode.game.cover[a][b] == 0 else 1 for b in range(currentNode.game.m)] for a in range(currentNode.game.n) ]
        add_flags = [flag for flag in currentNode.game.flagged]
        the_path.append((cov, [], add_flags))
        if currentNode.is_final_bad(): #daca nodul reprezinta o configuratie compromisa, nu mai e extins
          cov = [[0 if currentNode.parent.game.cover[a][b] == 0 else 1 for b in range(currentNode.game.m)] for a in range(currentNode.game.n) ]
          cov = currentNode.parent.game.cover
          currentNode.parent.game.flagged.append((currentNode.x, currentNode.y))
          add_flags = [flag for flag in currentNode.parent.game.flagged]
          the_path.append((cov, [], add_flags))
          for itr in open_set:
            change_node = get_key(visited_config, keys, itr[1])
            change_node.game.flagged.append((currentNode.x, currentNode.y))
          mistakes += 1
          continue
        closed_set.append(currentNode.game.cover)
        n = currentNode.game.n
        m = currentNode.game.m

        print("No of discoverd squares before")
        print(currentNode.game.discovered_squares)
        to_check = currentNode.game.solve()

        print("Cost:")
        print(currentNode.cost())
        print("No of discovered square:")
        print(currentNode.game.discovered_squares)
        add_flags = [flag for flag in currentNode.game.flagged]
        cov = [[0 if currentNode.game.cover[a][b] == 0 else 1 for b in range(currentNode.game.m)] for a in range(currentNode.game.n) ]
        the_path.append((cov, to_check, add_flags))
        if currentNode.game.flagged == currentNode.game.number_of_bombs:
          for i in range(n):
            for j in range(m):
              if currentNode.game.cover[i][j] == 0 and currentNode.game.table[i][j] != -1:
                currentNode.game.opened_cell(i,j)
                add_flags = [flag for flag in currentNode.game.flagged]
                cov = [[0 if currentNode.game.cover[a][b] == 0 else 1 for b in range(currentNode.game.m)] for a in range(currentNode.game.n) ]
                the_path.append((cov, [], add_flags))
        if currentNode.is_final_good():
          return currentNode, mistakes

        #print("A AJUNS AICI")
        print("Open set", open_set)
        if len(to_check) == 0: #daca nu mai exista celule acoperite si nemarcate, se va crea un nou nod printr-o miscare random pe restul tablei
          print("Cautare oprita, se alege random")
          for x in range(n):
            for y in range(m):
              if (x,y) in currentNode.game.flagged or currentNode.game.cover[x][y] == 1:
                continue
              newNode = copy.deepcopy(currentNode)
              newNode.parent = currentNode
              newNode.path.append(currentNode)
              newNode.moves = currentNode.moves + 1
              newNode.x = x
              newNode.y = y
              opened_cells = newNode.game.opened_cell(x, y)
              if newNode.is_final_bad():   #apasarile de mina sunt inregistrate
               add_flags = [flag for flag in newNode.game.flagged]
               cov = [[0 if newNode.game.cover[a][b] == 0 else 1 for b in range(newNode.game.m)] for a in range(newNode.game.n) ]
               the_path.append((cov, [], add_flags))
               add_flags = [flag for flag in newNode.parent.game.flagged]
               cov = [[0 if newNode.parent.game.cover[a][b] == 0 else 1 for b in range(newNode.game.m)] for a in range(newNode.game.n) ]
               the_path.append((cov, [], add_flags))
               currentNode.game.flagged.append((newNode.x, newNode.y)) #retinem pozitia minei apasate
               currentNode.game.solve()
               continue
              heappush(open_set, (currentNode.cost() - 1, newNode.game.cover))
              visited_config.update({newNode: newNode.game.cover})
              break



        for cell in to_check: #sunt create noi noduri pe baza celulelor care nu sunt sigure
           print("Se creeaza noi noduri")
           x = cell[0]
           y = cell[1]
           flags, saves = currentNode.game.sim_solve(x, y)
           print("New flags:")
           print(x,y)
           print(flags)
           if flags == [-1]:
            continue
           newNode = copy.deepcopy(currentNode)
           newNode.parent = currentNode
           newNode.path.append(currentNode)
           newNode.moves = currentNode.moves + 1
           newNode.x = x
           newNode.y = y
           opened_cells = newNode.game.opened_cell(x, y)
           #keys = list(visited_config.keys())
           values = list(visited_config.values())
           if newNode.game.cover in values:
            continue
           #key = get_key(visited_config, keys, newNode.game.cover)
           print(currentNode.cost())
           if newNode.heuristic == 2:
             heappush(open_set, (currentNode.cost() - len(flags) - len(saves) - 1, newNode.game.cover))
           if newNode.heuristic == 4:
             weight = 1 + currentNode.game.calc_weight(newNode.x, newNode.y)
             heappush(open_set, (currentNode.cost() - weight * (len(flags) - len(saves) - 1), newNode.game.cover))
           visited_config.update({newNode: newNode.game.cover})

In [8]:
def new_generate_game(n, m, number_of_bombs):
    table = [[0 for j in range(m)] for i in range(n)]
    cover = [[0 for j in range(m)] for i in range(n)]
    for bomb in range(number_of_bombs):
        x = random.randint(0, n - 1)
        y = random.randint(0, m - 1)
        while table[x][y] == -1:
            x = random.randint(0, n - 1)
            y = random.randint(0, m - 1)
        table[x][y] = -1
    for x in range(n):
        for y in range(m):
            if table[x][y] == -1:
                l1 = max([0, x - 1])
                l2 = min([n - 1, x + 1])
                c1 = max([0, y - 1])
                c2 = min([m - 1, y + 1])
                for a in range(l1, l2 + 1):
                   for b in range(c1, c2 + 1):
                     if table[a][b] != -1:
                        table[a][b] += 1
    start_game = Minesweeper(n, m, number_of_bombs, 0, False, table, cover, number_of_bombs, [], [])
    return start_game

In [8]:
game1 = new_generate_game(9, 9, 10) # generarea jocului de minesweeper prin tabla initiala
#game2 = generate_game(16, 16, 40)
#game3 = generate_game(16, 30, 99)

game = game1
node = Node(None, [], 2, game, 0, None, None) #crearea nodului radacina
node.game.print_table()
x = random.randint(0, node.game.n - 1) #alegerea celulei de start
y = random.randint(0, node.game.m - 1)
cov = [[0 if node.game.cover[a][b] == 0 else 1 for b in range(node.game.m)] for a in range(node.game.n) ]
print(cov)
the_path = [(cov, [])]
while node.game.table[x][y] == -1: #ne asiguram ca celula aleasa in start nu este una cu mina
      x = random.randint(0, node.game.n - 1)
      y = random.randint(0, node.game.m - 1)
start = time.time()
node.game.opened_cell(x,y)
print(cov)
print("PATH", the_path)
win, mistakes = new_a_star_v2(node)
end = time.time()
i = 1
for node in win.path: #reconstituirea drumului parcurs de algoritm
    print("Nodul ", i, node)
    i = i + 1
    node.game.print_cover()
    print('\n')
print("Nodul ", i)
win.game.print_cover()
win.game.print_table()
print(mistakes)
print("The time of execution of above program is :",
      (end-start) * 10**3, "ms")

for cover in the_path:
  for row in cover[0]:
    print(row)
  print('\n')


[0, 0, 0, 1, 2, 2, 2, -1, 2]
[0, 0, 0, 1, -1, -1, 3, 3, -1]
[0, 0, 0, 1, 2, 3, -1, 2, 1]
[0, 0, 0, 0, 0, 2, 2, 2, 0]
[0, 0, 0, 0, 0, 1, -1, 1, 0]
[1, 1, 1, 0, 0, 1, 1, 1, 0]
[1, -1, 1, 0, 0, 0, 0, 0, 0]
[2, 2, 2, 1, 1, 1, 1, 1, 1]
[1, -1, 1, 1, -1, 1, 1, -1, 1]
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0]]
PATH [([[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0,

In [15]:
avg_time = 0
avg_moves = 0
avg_mistakes = 0

for i in range(5):
  the_path = []
  game = new_generate_game(16, 16, 40)
  node = Node(None, [], 2, game, 0, None, None)
  x = random.randint(0, node.game.n - 1)
  y = random.randint(0, node.game.m - 1)
  while node.game.table[x][y] == -1: #ne asiguram ca celula aleasa in start nu este una cu mina
      x = random.randint(0, node.game.n - 1)
      y = random.randint(0, node.game.m - 1)
  node.game.opened_cell(x,y)
  start = time.time()
  win, mistakes = new_a_star_v2(node)
  end = time.time()
  avg_time += (end - start) * 10 ** 3
  avg_moves += len(the_path)
  avg_mistakes += mistakes

print("Average time of execution :", avg_time/5)
print("Average number of moves: ", avg_moves/5)
print("Average number of mistakes: ", avg_mistakes/5)

No of discoverd squares before
1
Cost:
255
No of discovered square:
1
Open set []
Se creeaza noi noduri
New flags:
2 1
[]
255
Se creeaza noi noduri
New flags:
2 2
[]
255
Se creeaza noi noduri
New flags:
2 3
[]
255
Se creeaza noi noduri
New flags:
3 1
[]
255
Se creeaza noi noduri
New flags:
3 3
[]
255
Se creeaza noi noduri
New flags:
4 1
[]
255
Se creeaza noi noduri
New flags:
4 2
[]
255
Se creeaza noi noduri
New flags:
4 3
[]
255
[<__main__.Node object at 0x7b5b331a39a0>, <__main__.Node object at 0x7b5ba6201420>, <__main__.Node object at 0x7b5b13364ca0>, <__main__.Node object at 0x7b5b122eee30>, <__main__.Node object at 0x7b5b122ed2a0>, <__main__.Node object at 0x7b5b122ede70>, <__main__.Node object at 0x7b5b122efc70>, <__main__.Node object at 0x7b5b122ece50>, <__main__.Node object at 0x7b5bb98b1630>]
No of discoverd squares before
2
Cost:
254
No of discovered square:
2
Open set [(254, [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 

KeyboardInterrupt: ignored

INTERFATA

In [ ]:
import tkinter as tk



def show_board(table, cover, n, m):
    root = tk.Tk()
    root.title("Minesweeper")



    canvas = tk.Canvas(root, width=n * 30, height=m * 30)
    canvas.pack()

    for x in range(n):
        for y in range(m):
            if cover[x][y] == 0:
              canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='grey')
              canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='', fill='grey')
            else:
              if table[x][y] == -1:
                 canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='red')
                 canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='B', fill='black')
              else:
                  canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='white')
                  if table[x][y] == 0:
                    canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text=' ', fill='black')
                  else:
                    canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text=str(table[x][y]), fill='black')

    root.mainloop()


show_board(win.game.table, win.game.cover, win.game.n, win.game.m)


TclError: ignored

In [ ]:
def show_board():
    global p
    if len(p) == 0:
        return
    cover = p[0][0]
    to_check = p[0][1]
    flags = p[0][2]
    p = p[1:]

    for x in range(n):
        for y in range(m):
            if cover[x][y] == 0:
              if (x,y) in flags:
                  canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='pink')
                  canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='F', fill='black')
              else:
                if (x,y) in to_check:
                  canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='green')
                  canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='', fill='green')
                else:
                    canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='grey')
                    canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='', fill='grey')
            else:
              if table[x][y] == -1:
                 canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='red')
                 canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text='B', fill='black')
              else:
                  canvas.create_rectangle(x * 30, y * 30, (x + 1) * 30, (y + 1) * 30, fill='white')
                  if table[x][y] == 0:
                    canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text=' ', fill='black')
                  else:
                    canvas.create_text((x + 0.5) * 30, (y + 0.5) * 30, text=str(table[x][y]), fill='black')
    root.after(1000, show_board)

show_board()
root.mainloop()